<a href="https://colab.research.google.com/github/gustavococoo/Python-3/blob/main/%C3%A1rvore_de_decis%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importando as bibliotecas necessárias:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier,export_graphviz
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np

In [22]:
# Carregando a base de dados:
df_diabetes = pd.read_csv('/content/diabetes.csv', sep = ';')
df_diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
# Informaçoes sobre a base:
df_diabetes.info()



In [44]:
X =  df_diabetes[['Glucose','BloodPressure', 'BMI',	'DiabetesPedigreeFunction', 'Age']]
y = df_diabetes['Outcome']
# Dividindo os dados em treino e teste:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3)

In [45]:
# Verificando as formas dos dados:
X_train.shape,X_test.shape
((537, 8), (231, 8))

((537, 8), (231, 8))

In [46]:
y_train.shape,y_test.shape
((537,), (231,))

((537,), (231,))

In [47]:
# Instânciando o objeto classificador:
clf = DecisionTreeClassifier()

In [48]:
# Treinando o modelo de arvore de decisão:
clf = clf.fit(X_train,y_train)

In [49]:
# Verificando as features mais importantes para o modelo treinado:
clf.feature_importances_


array([0.30191775, 0.12924302, 0.16855607, 0.20830841, 0.19197475])

In [50]:
for feature,importancia in zip(df_diabetes.columns,clf.feature_importances_):
    print("{}:{}".format(feature, importancia))


Pregnancies:0.3019177474780518
Glucose:0.12924301757035772
BloodPressure:0.16855607071646858
SkinThickness:0.2083084141488411
Insulin:0.19197475008628084


In [ ]:
resultado = clf.predict(X_test)
resultado

In [52]:
from sklearn import metrics
print(metrics.classification_report(y_test,resultado))

              precision    recall  f1-score   support

           0       0.75      0.74      0.74       153
           1       0.50      0.51      0.51        78

    accuracy                           0.66       231
   macro avg       0.62      0.63      0.62       231
weighted avg       0.66      0.66      0.66       231



In [ ]:
# Renderizando a árvore de forma gráfica:
!pip install ipywidgets
!pip3 install graphviz
!pip3 install pydot
import pydot
import graphviz

dot_data = export_graphviz( 
         clf, 
         out_file=None,
         feature_names=df_diabetes.drop('Outcome',axis=1).columns,
         class_names=['0','1'],  
         filled=True, rounded=True,
         proportion=True,
         node_ids=True,
         rotate=False,
         label='all',
         special_characters=True
        )  
graph = graphviz.Source(dot_data)  
graph

In [18]:
# Renderizando a árvore de forma interativa:
from ipywidgets import interactive
from IPython.display import SVG,display
from graphviz import Source

# feature matrix
X,y = df_diabetes.drop('Outcome',axis=1),df_diabetes['Outcome']

# feature labels
features_label = df_diabetes.drop('Outcome',axis=1).columns

# class label
class_label = ['0','1']


def plot_tree(crit, split, depth, min_samples_split, min_samples_leaf=0.2):
    estimator = DecisionTreeClassifier(
           random_state = 0 
          ,criterion = crit
          ,splitter = split
          ,max_depth = depth
          ,min_samples_split=min_samples_split
          ,min_samples_leaf=min_samples_leaf
    )
    estimator.fit(X, y)
    graph = Source(export_graphviz(estimator
      , out_file=None
      , feature_names=features_label
      , class_names=class_label
      , impurity=True
      , filled = True))
    display(SVG(graph.pipe(format='svg')))
    return estimator

inter=interactive(plot_tree 
   , crit = ["gini", "entropy"]
   , split = ["best", "random"]
   , depth=[1,2,3,4,5,10,20,30]
   , min_samples_split=(1,5)
   , min_samples_leaf=(1,5))

display(inter)

interactive(children=(Dropdown(description='crit', options=('gini', 'entropy'), value='gini'), Dropdown(descri…

In [ ]:
# Visualizando as fronteiras criadas pela arvore:
from mlxtend.plotting import plot_decision_regions
import matplotlib.pyplot as plt

def visualize_fronteiras(msamples_split,max_depth):
    X = df_diabetes[['Glucose','Insulin']].values
    y = df_diabetes.Outcome.values
    clf = DecisionTreeClassifier(min_samples_split=msamples_split,max_depth=max_depth)
    tree = clf.fit(X, y)

    plt.figure(figsize=(16,9))
    plot_decision_regions(X, y, clf=tree, legend=2)

    plt.xlabel('Glucose')
    plt.ylabel('Insulin')
    plt.title('Decision Tree')
    plt.show()

# Chamando a função criada anteriormente:

visualize_fronteiras(2,max_depth=30)

In [ ]:
# Chamando a função criada anteriormente:

visualize_fronteiras(10,max_depth=8)


In [ ]:
# Navegando na estrutura da arvore de decisão:
estimator = DecisionTreeClassifier(max_depth=3,min_samples_split=2,min_samples_leaf=2)
estimator.fit(X, y)
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')
n_nodes = estimator.tree_.node_count
children_left = estimator.tree_.children_left
children_right = estimator.tree_.children_right
feature = estimator.tree_.feature
threshold = estimator.tree_.threshold

print("Número de nós: {}".format(n_nodes))

In [ ]:
node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
is_leaves = np.zeros(shape=n_nodes, dtype=bool)
stack = [(0, -1)]
while len(stack) > 0:
    node_id, parent_depth = stack.pop()
    node_depth[node_id] = parent_depth + 1

    if (children_left[node_id] != children_right[node_id]):
        stack.append((children_left[node_id], parent_depth + 1))
        stack.append((children_right[node_id], parent_depth + 1))
    else:
        is_leaves[node_id] = True

print("\nA arvore binária tem %s nós e a seguinte estrutura: \n"
      % n_nodes)
for i in range(n_nodes):
    if is_leaves[i]:
        print("%snó=%s (nó folha)." % (node_depth[i] * "\t", i))
    else:
        print("%snó=%s (nó teste): vai para o nó %s se o valor do atributo %s <= %s \n se não, vai para o "
              "nó %s."
              % (node_depth[i] * "\t",
                 i,
                 children_left[i],
                 df_diabetes.columns[feature[i]],
                 threshold[i],
                 children_right[i],
                 ))